In [ ]:
import numpy as np

from scipy.stats import multivariate_normal

import matplotlib.pyplot as plt


The probability distribution we wish to draw samples from.

$$ P(x, y) \propto \exp\left( \frac{-\left(\sqrt{x^2+y^2}-r_0\right)^2}{2}\right)  $$

In [ ]:
def P(x, r0=5, w=1):
    """
    Target distribution, a ring/annulus in the 2D plane
    
    INPUTS
    ------
    r0, w: floats
        parameters of the distribution, the ring radius and thickness
    
    RETURNS
    -------
    Px: float
        the value of the PDF, P(x)
    """
    r = np.sqrt(x[0]**2+x[1]**2)
    return np.exp(-0.5*((r-r0)/w)**2)


In [ ]:
def Q(x, cov=np.identity(2)):
    """
    Proposal distribution, a 2D Gaussian centred on the current point
    
    INPUTS
    ------
    x: array
        the current point
    
    RETURNS
    -------
    X: array
        proposed point
    """
    X = multivariate_normal(x, cov).rvs()
    return X


# A simple implementation of the Metropolis-Hastings MCMC algorithm

In [ ]:
# Initial starting point
x0 = np.array([0, 0])

# Create a chain of the points we have visited
chain = [x0]

# Number of iterations (or steps) to take
num_iterations = 5000

# iterate the algorithm
for i in range(num_iterations):
    
    # current point is last point in the chain
    x = chain[-1]
    
    # proposed point
    y = Q(x)
    
    # acceptance ratio
    alpha = P(y)/P(x)
    
    # random number determines acceptance
    u = np.random.uniform()
    
    if u<alpha: # accept the proposed point
        x_new = y
    else: # reject the proposed poont
        x_new = x
        
    # append new point to the chain
    chain.append(x_new)

Let's plot the first few points in the MCMC chain.

In [ ]:
samples = np.array(chain)[0:100,:]

ax = plt.gca()
ax.set_aspect(1.)

ax.plot(samples[:,0], samples[:,1])

ax.set_xlim(-7,7)
ax.set_ylim(-7,7)

ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")

ax.set_aspect(1)

plt.show()

Let's plot a histogram of chain (with a burnin portion removed and with thinning).|

In [ ]:
import corner 

samples = np.array(chain)

# remove a burnin portion of the chain and thin the rest of the chain
burnin = 100
thin = 10
thin_samples = samples[burnin::thin]

# plot histogram
corner.corner(samples, labels=[r'$x$', r'$y$'])

plt.show()

### Try rerunning the above with (i) more iterations (ii) different amounts of burnin and thinning.

# MCMC Sampling Using Emcee 

https://emcee.readthedocs.io/en/stable/

In [ ]:
import emcee

ndim = 2 # our problem is 2 dimensional
nwalkers = 10 # instead of a single chain this algorithm uses several

x0 = np.random.randn(nwalkers, ndim) # choose random starting points 

# most sampling algorithms take the log of the PDF as input
def log_P(x):
    return np.log(P(x))

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_P) # setup the sampler

state = sampler.run_mcmc(x0, 100) # run 100 burn in iterations

sampler.reset() 

sampler.run_mcmc(state, 5000) # run algorithm for 10000 iterations

samples = sampler.get_chain(flat=True, thin=20) # get the thinned mcmc chain

corner.corner(samples, labels=[r'$x$', r'$y$'])

plt.show()

# Nested Sampling Using Dynesty



In [ ]:
ndim = 2

def prior_transform(u):
    """Transforms the uniform random variable `u ~ Unif[0., 1.)`
    to the parameter of interest `x ~ Unif[-10., 10.)`."""

    x = 2. * u - 1.  # scale and shift to [-1., 1.)
    x *= 10.  # scale to [-10., 10.)

    return x

def log_likelihood(x):
    r0, w = 5., 1.
    r = np.sqrt(x[0]**2+x[1]**2)
    return -0.5*((r-r0)/w)**2

from dynesty import NestedSampler

sampler = NestedSampler(log_likelihood, prior_transform, ndim, nlive=1024, bound='multi', sample='slice')

sampler.run_nested(dlogz=0.01)

In [ ]:
corner.corner(sampler.results.samples, labels=['x', 'y'])
plt.show()